In [ ]:
#Importing all packages
import selenium
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re
from bs4 import BeautifulSoup
import urllib
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import WebDriverException
import time
import csv

In [ ]:
#Calling Chrome driver
chrome_path = "C:/Users/sangh/Documents/Anaconda_files/Healthcare/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

In [ ]:
#Creating different lists to extract info from each page
doctor_names=[]
school=[]
year = []

In [ ]:
#Defining fucntion for extracting demographics
def demoInfo(url):
    
    #Getting all the individual doctor page links
    list_of_hrefs = []
    
    doctor_list = driver.find_elements_by_id("doctor-list")
    time.sleep(1)

    for block in doctor_list:
        elements = block.find_elements_by_class_name("search-item-doctor-link")
        for el in elements:
            list_of_hrefs.append(el.get_attribute("href"))

    #print (list_of_hrefs)
    
    #Removing all unwanted characters in the link list 
    try:
        regex = re.compile(r'^tel:')
        href=[x for x in list_of_hrefs if not regex.match(x)]
    except:
        list_of_hrefs=list(filter(None,list_of_hrefs))
        regex = re.compile(r'^tel:')
        href=[x for x in list_of_hrefs if not regex.match(x)]
    finally:
        print (len(href))
        append_str = '#credentials'
        list_new = [sub + append_str for sub in href]
        print(list_new)
        print(len(list_new))
    
    for j in range(len(list_new)):
        time.sleep(2)
        driver.get(list_new[j])     
        
        time.sleep(5)
        
        try:
            name=driver.find_element_by_xpath("//h1[@itemprop='name']")
            doctor_names.append(name.text)
        except:
            name.append("Unknown")
        
        try:
            s=driver.find_element_by_xpath("//*[@id='left-content']/span/div[2]/span/ul/li/span[1]")
            school.append(s.text)
        except:
            school.append("Unknown")    
        
        try:
            y=driver.find_element_by_xpath("//*[@id='left-content']/span/div[2]/span/ul/li/span[2]")
            year.append(y.text)
        except:
            year.append("Unknown")
            
    print (len(doctor_names))
    print (len(school))   
    print (len(year))

In [ ]:
pageLinks=[]
#Setting initial URL
url=driver.get("https://www.ratemds.com/best-doctors/il/?specialty=gynecologist-obgyn&page=1")
source=driver.current_url
pageLinks.append(source)
time.sleep(2)

#Loop to get all the pageLinks for Doctors
for i in range(112):
    try:
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="doctor-list"]/div/nav/ul/li[11]/a/span/i'.format(i)).click()
        source = driver.current_url
        pageLinks.append(source)
    except ConnectionResetError:
        continue
    except NoSuchElementException:
        continue
    except:
        continue
print(len(pageLinks))

In [ ]:
len(pageLinks)

In [ ]:
pageLinks.remove(None)
pageLinks.append(driver.current_url)
seen = set()
result = []
for item in pageLinks:
    if item not in seen:
        seen.add(item)
        result.append(item)

In [ ]:
#Getting all the demographic info
for i in range(len(pageLinks)):
    url=driver.get(pageLinks[i])
    print (pageLinks[i])
    time.sleep(1)
    demoInfo(url)

In [ ]:
#Checking length of lists 
print (len(doctor_names))
print (len(school))
print(len(year))

In [ ]:
print(school)

In [ ]:
print(year)

In [ ]:
#Output to csv file
reviewDict={'Name':doctor_names, 'University':school, 'Year':year}
dataframe=pd.DataFrame(data=reviewDict)
dataframe
dataframe.to_csv('RateMD_Demographics.csv', header=True, mode='a')